# OPM Coords to WGS84

OPM coordinates not originally in WGS84; convert and save file with OPM UID and lat/lon

## Setup

In [5]:
# install.packages("rgeos", dep = T)

also installing the dependencies ‘maptools’, ‘XML’, ‘rgdal’


Warning message in install.packages("rgeos", dep = T):
“installation of package ‘rgdal’ had non-zero exit status”
Warning message in install.packages("rgeos", dep = T):
“installation of package ‘rgeos’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [6]:
# Load packages
library(tidyverse)
library(haven)
library(rgeos)

# Set up sagemaker
library(reticulate)
sagemaker <- import('sagemaker')
session <- sagemaker$Session()

# s3 bucket name
bucket = "worldbank-pakistan-data"
local_dir = '/home/ec2-user/SageMaker/'

ERROR: Error in library(rgeos): there is no package called ‘rgeos’


## Load Data

In [ ]:
session$download_data(path = local_dir,
              bucket = bucket,
              key_prefix = 'OPM/RawData - PII/GPS_uid_crosswalk.dta')

session$download_data(path = local_dir,
              bucket = bucket,
              key_prefix = 'GADM/RawData/gadm36_PAK_0_sp.rds')

In [ ]:
bisp_coords <- read_dta(file.path(local_dir, "GPS_uid_crosswalk.dta"))
pak_adm0 <- readRDS(file.path(local_dir, "gadm36_PAK_0_sp.rds"))

## Prep Data

In [2]:
bisp_coords <- bisp_coords %>%
  filter(!is.na(GPSN)) %>%
  
  mutate(latitude = get_lat_lon(GPSN),
         longitude = get_lat_lon(GPSE),
         uid = uid %>% as.numeric()) %>%
  
  dplyr::select(uid, latitude, longitude) %>%
  
  filter(latitude <= 100,
         longitude <= 100)

## Restrict to Coordinates in Pakistan
bisp_coords_sdf <- bisp_coords
coordinates(bisp_coords_sdf) <- ~longitude+latitude
crs(bisp_coords_sdf) <- CRS("+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0")

dist_to_pak <- gDistance(bisp_coords_sdf, pak_adm0, byid=T) %>% as.vector()

bisp_coords <- bisp_coords[dist_to_pak == 0,]

ERROR: Error in filter(., !is.na(GPSN)): object 'bisp_coords' not found


## Export

In [ ]:
write.csv(bisp_coords, file.path(local_dir, "GPS_uid_crosswalk.csv"), row.names = F)

session$upload_data(path       = file.path(local_dir, "GPS_uid_crosswalk.csv"), 
                    bucket     = bucket, 
                    key_prefix = file.path("OPM", "FinalData - PII"))